# Trabajo final — Generación de música clásica con RNN
## Ander Aguinaga San Sebastián — MAIS 3.º A
#### 2023/01/16

### Importamos las librerías necesarias
Para lo relacionado con el modelo, necesitamos importar Tensorflow, Keras y Numpy, como de costumbre. Utilizamos también la librería `glob` para acceder, como veremos a continuación, a ficheros dentro de una carpeta. Por último, la librería `music21` nos ayudará a manejar ficheros MIDI.

In [ ]:
import tensorflow as tf
import numpy as np
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Activation
from keras import Sequential
from keras.callbacks import ModelCheckpoint

import glob

from music21 import converter, instrument, note, chord, stream

print("Número de GPU disponibles: ", len(tf.config.list_physical_devices('GPU')))

### Importamos los datos
Los datos son notas y acordes que leeremos de los archivos `.mid` que tengamos en nuestra carpeta de datos de entrenamiento. De cada uno de estos archivos, extraeremos los elementos de tipo `note.Note` o `chord.Chord` (de `music21`) que encontremos (hay más objetos además de esos, como la clave o la fórmula de compás, pero estos no nos interesan para lo que queremos hacer). Los guardaremos como cadenas de texto que contienen de forma legible la nota que tienen asociada (las notas se dan separadas por punto en el caso de los acordes). Por ejemplo, si en el MIDI se encuentra un la central, se añadirá al _array_ `notes` la cadena _A4_.

In [ ]:
def import_data(folder):
    notes = []
    for file in glob.glob(folder + "/*"):
        midi = converter.parse(file)
        notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    global n_vocab
    global pitches
    pitches = sorted(set(notes))
    n_vocab = len(pitches)
    return notes

### Preparamos las secuencias

La idea es entrenar nuestro modelo dándole como _input_ una secuencia de notas o acordes, y generando como _output_ un _array_ que nos dirá la probabilidad de que la siguiente nota o acorde sea una dada. El _input_ estará normalizado para asegurar la mayor eficacia posible en el modelo.

In [ ]:
def prepare_sequences(notes):
    sequence_length = 100
    # get all pitch names
    global pitches
    pitches = sorted(set(notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitches))

    network_input = []
    network_output = []
    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    
    n_patterns = len(network_input)
    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)
    network_output = np_utils.to_categorical(network_output)

    return network_input, network_output

### Definimos la arquitectura de la red

Será una red neuronal recurrente (RNN), con varias capas LSTM que ayudarán a encontrar patrones en los datos. El motivo de utilizar tantas capas `Dropout` es que queremos eviatar el _overfitting_, propenso a aparecer en este tipo de situaciones. El modelo acaba con una capa densa de `n_vocab`, el número de notas y acordes distintos, que sería el equivalente al tamaño del abecedario en un modelo de generación de texto.

In [ ]:
def create_model(network_input):
    model = Sequential()
    model.add(LSTM(
        256,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True
    ))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))

    return model

### Preparamos el modelo
Compilamos utilizando `categorical_crossentropy` como función _loss_, pues es un problema de tipo categórico, y el optimizador RMS Prop, que es el que mejor ha funcionado según las pruebas que he estado haciendo.

In [ ]:
def prepare_model(network_input):
    model = create_model(network_input)
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

### Entrenamos el modelo
Al entrenar, guardamos los pesos de cada época en que el _loss_ haya disminuido en comparación a la época anterior, de esta forma siempre podremos recuperar los pesos de las mejores épocas sin necesidad de gastar memoria almacenando los pesos de absolutamente todas ellas.

In [ ]:
def train_model(model, network_input, network_output, epochs, batch_size, filepath = "weights/e-{epoch:02d}-l-{loss:.4f}.hdf5"):
    checkpoint = ModelCheckpoint(
        filepath, monitor='loss', 
        verbose=0,        
        save_best_only=True,        
        mode='min'
    )    
    callbacks_list = [checkpoint]     
    model.fit(network_input, network_output, epochs=epochs, batch_size=batch_size, callbacks=callbacks_list)

### Generamos notas
Primero tomamos al azar una de las secuencias de notas que habíamos preparado antes para el entrenamiento, y la guardamos en `pattern`. Partiendo de ahí, comenzamos a predecir las notas siguientes, y utilizando las nuevas notas generadas en la secuencia _input_ para la siguiente predicción. Ya que el modelo predice un _array_ de probabilidades, la nota que nos interesa corresponde al índice de la probabilidad más alta.

Con esta función acabamos generando un array de _strings_ que contienen el nombre de la nota, como hemos visto antes.

In [ ]:
def generate_notes(model, network_input, num_notes):
    start = np.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitches))
    pattern = network_input[start]*n_vocab
    prediction_output = []
    
    for note_index in range(num_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]
    
    return prediction_output

### Generamos el MIDI
Convertimos las notas generadas a objetos de `music21` que van a poder ser exportados a un fichero MIDI. Si se ha generado un acorde, se extrae al MIDI una nota para cada una de las que había en el acorde, sin modificar el _offset_ para que se escuchen a la vez. Si se ha generado una nota, se extrae esa nota al MIDI, a no ser que coincida con la anterior, en cuyo caso se actualizará el _offset_ sin extraer una nota nueva. De esta forma, se da la sensación de que la duración de las notas varía, en lugar de escuchar notas repetidas.

In [ ]:
def create_midi(filename, prediction_output):
    offset = 0
    output_notes = []
    # create note and chord objects based on the values generated by the model
    last_pattern = None
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        elif(last_pattern != pattern):
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        last_pattern = pattern
        # increase offset each iteration so that notes do not stack
        offset += 0.5
    
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp=filename)

Esta función simplemente se encarga de llamar a las dos anteriores.

In [ ]:
def generate_midi(model, network_input, num_notes, filename = "output.mid"):
    prediction_output = generate_notes(model, network_input, num_notes)
    print(prediction_output)
    create_midi(filename, prediction_output)

### Llamamos a las funciones
Ya que todo el código que hemos visto estaba en funciones aisladas, ahora solo debemos llamarlas en orden.

In [ ]:
notes = import_data("bach1")
network_input, network_output = prepare_sequences(notes)
model = prepare_model(network_input)
model.load_weights('weightsBach.hdf5')
#train_model(model, network_input, network_output, 20, 64)
generate_midi(model, network_input, 500)

### Conclusión
En un principio quise intentar generar música de forma algo más sofisticada, tomando en cuenta también la duración de las notas, cosa que en este proyecto no he hecho. Complicaba bastante las cosas, y creo que es mejor empezar con algo más simple, y avanzar a eso más complejo con más tiempo en un futuro. Además, como expliqué arriba, evitando insertar dos notas iguales una detrás de otra, he acabado dando el efecto de que sí varía la duración, aunque internamente no sea del todo así.

No sé exactamente cómo interpretar los resultados, pues han ocurrido todo tipo de cosas. He probado con dos tres conjuntos de datos: dos con piezas de Bach y uno con piezas de Debussy.

Cuando entreno con piezas de Debussy (carpeta _debussy_), la música generada suele ser monótona y con muchos acordes. La obra de Debussy es impresionista, y por su naturaleza, además de tener muchos más acordes, creo que es más difícil de predecir. Por eso, la mejor solución que encuentra el modelo es acabar reproduciendo en bucle un mismo acorde (_debussy1.mid_).

Bach es de estilo barroco, sus preludios y fugas tienen mucho contrapunto, son piezas más _calculadas_, más _matemáticas_, y en ese sentido puede hacérsele más fácil a una IA generar música así.

El primer conjunto de datos de Bach (carpeta _bach1_) tenía tres MIDIs, cada uno conteniendo un preludio y una fuga. Pensé que sería un conjunto de datos provisional, pues tenía pensado buscar más MIDIs y hacer un entrenamiento mejor. Los resultados fueron bastante positivos: aunque a veces se note el _overfitting_ porque genera música demasiado similar a las piezas originales (_bach2.mid_ se parece mucho al preludio V), en general reproduce algo escuchable (_bach1.mid_), que a veces parece algo aleatorio, pero por lo menos tiene más movimiento que con Debussy, donde se quedaba estancado.

El segundo conjunto de datos de Bach (carpeta _bach2_) tenía 19 MIDIs, cada uno conteniendo un preludio o una fuga. Para mi sorpresa, obtuve resultados peores, se quedaba estancado muchísimo más: como con Debussy, pero sin tantos acordes. Es cierto que no se notan copias descaradas, pero reproduce una pieza final monótona y que nadie querría escuchar (_bach3.mid_). Además, si seguía entrenándolo, incluso sin llegar a tantos entrenamientos que con el _dataset_ anterior, ya se quedaba completamente estancado y producida MIDIs de una única nota (_bach4.mid_).

También probé con distintas arquitecturas de red, modificando el ratio de _dropout_, quitando capas _LSTM_, modificando el número de neuronas, etc., y al mínimo cambio acababa obteniendo resultados considerablemente peores, que producían notas estancadas.

En definitiva, ¡he obtenido resultados peores cada vez que intentaba mejorar lo que tenía! Pero he aprendido mucho trabajando en ello. :)